In [ ]:
# import comet_ml in the top of your file
from comet_ml import Experiment
# Add the following code anywhere in your machine learning file
experiment = Experiment(api_key="Go4XzK6f4G10W9jFaWiHg6JP2",
                        project_name="tbm", workspace="twofyw")

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from config import *
config = Config('rnn-param-tune', debug=False, sl=30, cont_names=Config.all_columns(), postpond=0, fn_feather='tbmData/feather/all_columns-full-30-s-0.feather')

torch.cuda.set_device(config.gpu_start); torch.ones((1, 1)).cuda(); # wakeup gpu

# load data
cyc_cont = pd.read_feather(config.fn_feather)
npz = np.load(config.fn_np)
idx, stat_x, stat_y = npz['idx'], npz['stat_x'], npz['stat_y']

rnndb = MultiDeptTabularDataBunch.from_df('tmp', cyc_cont, config.dep_var, valid_idx=config.valid_idx_tile, bs=config.bs, num_workers=0)

record = False

In [ ]:
hyper_params = {
    'n_hidden': 100,
    'n_layers': 3,
    'hidden_p': 0.3,
    'input_p': 0.6,
    'weight_p': 0.5,
    'layers': [3*config.n_cont, 5, 2],
    'drops': [0.2] * 3,
    'alpha': 2.,
    'beta': 1.,
    'clip': 0.5,
#     'loss_func' = MSELossFlat()
    'loss_func': l1
}
# experiment.log_parameters(hyper_params)
def get_cont_model(hyper_params):
    rnn_enc = ContModel1(1, config.n_cont, hyper_params['n_hidden'], hyper_params['n_layers'], sl=config.sl,
                         hidden_p=hyper_params['hidden_p'], input_p=hyper_params['input_p'], embed_p=0, weight_p=hyper_params['weight_p'])
    model = MultiInputSequentialRNN(rnn_enc, PoolingLinearClassifier(hyper_params['layers'], hyper_params['drops'])).cuda()

    metrics = [MAPD(stat_y)]
    learner = Learner(rnndb, model, loss_func=hyper_params['loss_func'], metrics=metrics, opt_func=optim.SGD)

    # learner.callback_fns += [ShowGraph, partial(SaveModelCallback, name='rnn0')]
    learner.callback_fns += [ShowGraph,]
    learner.callbacks += [TerminateOnNaNCallback()]
    learner.callbacks.append(RNNTrainer(learner, config.sl, alpha=hyper_params['alpha'], beta=hyper_params['beta']))

    learner.callback_fns.append(partial(GradientClipping, clip=hyper_params['clip']))
    learner.split(rnn_classifier_split)
    return learner
learner = get_cont_model(hyper_params)

In [ ]:
optimizer = Optimizer("Go4XzK6f4G10W9jFaWiHg6JP2")

In [ ]:
# Declare your hyper-parameters:
params = """
n_hidden integer [27, 100] [100]
"""
optimizer.set_params(params)

In [ ]:
suggestion = optimizer.get_suggestion()

In [ ]:
from comet_ml import Optimizer
optimizer = Optimizer("Go4XzK6f4G10W9jFaWiHg6JP2")
# Declare your hyper-parameters:
params = """
n_hidden integer [27, 100] [100]
"""
optimizer.set_params(params)

while True:
    # Get a suggestion
    suggestion = optimizer.get_suggestion()

    # Create a new experiment associated with the Optimizer
    experiment = Experiment(optimizer.api_key, project_name='suggestion')

    # Test the model
    learner = get_cont_model(hyper_params)
    score = fit(suggestion["x"])

    # Report the score back
    suggestion.report_score("accuracy",score)

In [ ]:
learner.unfreeze()
learner.lr_find(num_it=100)
learner.recorder.plot(1,1)

In [ ]:
with experiment.train():
    lr = 1e-0
    learner.fit_one_cycle(9, lr)